#Классификация слов по контексту

##Подгрузка импортов

In [100]:
pip install transformers

In [101]:
import numpy as np
import pandas as pd
import torch
import transformers
import json,io
from transformers import BertTokenizerFast, BertModel, Trainer, TrainingArguments

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

## Подготовка датасета

In [121]:
#максимальная длинна предложения
MAX_LENGTH = 120

In [122]:
def generate_bert_input(file_name,model_name):
    ###
    #input data:
    #
    #file_name: имя файла формата json c сэмплами вида [[[5, 10]], ["Если монах ведёт себя согласно со своими обетами, он не страшилище, не чуждое, почти враждебное нам существо, не живой труп, а наоборот, великий духовный друг наш и отец, носитель духовной благодатной жизни, молитвенник за нас пред Богом."], ["монах 0"], ["член религиозной общины, давший обет ведения аскетической жизни"], [1]]
    #model_name: имя bert модели
    #
    #return:
    #
    #лист картежей вида ((tokens_tensor, segments_tensors, offset_mapping, samp_position),(tokens_tensor, segments_tensors, offset_mapping, samp_position),label)
    #где tokens_tensor токенезированные слова определения или примера употребления в зависимости от картежа соответственно
    #где segments_tensors длина  определения или примера употребления в зависимости от картежа соответственно
    #где offset_mapping позиция ключеовго слова  определения(всегда первое) или примера употребления в зависимости от картежа соответственно
    #где samp_position позиция ключеовго слова из базы данных для определения(всегда первое) или примера употребления в зависимости от картежа соответственно
    ###
    data = []
    with io.open(file_name, 'r', encoding="utf-8-sig") as f:
        data = json.load(f)
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    samples_data = []
    i = 0
    for text in data:
        if i==800:
          break
        #получаем данные для примера употребления
        marked_text = text[1][0]
        tokenized_text = tokenizer(marked_text, return_offsets_mapping=True,max_length=MAX_LENGTH,padding='max_length',truncation=True)
        offset_mapping = tokenized_text['offset_mapping']
        indexed_tokens = tokenized_text['input_ids']
        segments_ids = [1] * len(tokenized_text['input_ids'])
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        samp_position = text[0][0]
        sample_data = (tokens_tensor, segments_tensors, offset_mapping, samp_position)

        #получаем данные для определения
        #берём само слово text[2][0][:-2] и его определение text[3][0] и строим текст вида
        #(слово - определение)
        definition = text[2][0][:-2]+" - "+text[3][0]
        def_positions = [0, len(text[2][0][:-2])]

        marked_text = definition 
        tokenized_text = tokenizer(marked_text, return_offsets_mapping=True,max_length=MAX_LENGTH,padding='max_length',truncation=True)
        offset_mapping = tokenized_text['offset_mapping']
        indexed_tokens = tokenized_text['input_ids']
        segments_ids = [1] * len(tokenized_text['input_ids'])
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        def_data = (tokens_tensor, segments_tensors, offset_mapping, def_positions)

        temp_data = ((def_data),(sample_data),text[4])
        samples_data.append(temp_data)
        i += 1
    return samples_data

In [123]:
class HyperonymDataset(torch.utils.data.Dataset):
  
  def __init__(self, dataset):
      self.dataset = dataset

  def __len__(self):
        return len(self.dataset)

  def __getitem__(self, idx):
        items = {
        "tokens_tensor_def": torch.tensor(self.dataset[idx][0][0]),
        "segments_tensors_def": torch.tensor(self.dataset[idx][0][1]),
        "offset_mapping_def": torch.tensor(self.dataset[idx][0][2]),
        "samp_position_def": torch.tensor(self.dataset[idx][0][3]),

        "tokens_tensor_samp": torch.tensor(self.dataset[idx][1][0]),
        "segments_tensors_samp": torch.tensor(self.dataset[idx][1][1]),
        "offset_mapping_samp": torch.tensor(self.dataset[idx][1][2]),
        "samp_position_samp": torch.tensor(self.dataset[idx][1][3]),

        "labels": torch.tensor(self.dataset[idx][2][0])
        }

        return items
        # items = {"dataset": self.dataset[idx]}
        # return items

##Модель

In [124]:
class SemanticSimilarityBertModel(torch.nn.Module):
    def __init__(self):
        super(SemanticSimilarityBertModel, self).__init__()

        #Подгружаем модель
        model_name = 'sberbank-ai/ruBert-base'
        self.model = BertModel.from_pretrained(model_name, output_hidden_states = True)

        #Файн-тьюниги
        self.biderectional_lstm = torch.nn.LSTM(input_size = 768*2, hidden_size = 768, bidirectional =True, batch_first = True)
       # self.AvgPool1D = torch.nn.AvgPool1d(768*2) 
        #self.MaxPool1D = torch.nn.MaxPool1d(768*2)
        #self.concatenate = torch.cat((self.AvgPool1D,self.MaxPool1D),dim = 1)
        self.Dropout = torch.nn.Dropout(0.3)
        self.Linear =  torch.nn.Linear(3072, 2)

    def forward(self, tokens_tensor_def,segments_tensors_def,offset_mapping_def,samp_position_def, tokens_tensor_samp,segments_tensors_samp,offset_mapping_samp,samp_position_samp, labels):
        # ex = [[[5, 10]], ["Если монах ведёт себя согласно со своими обетами, он не страшилище, не чуждое, почти враждебное нам существо, не живой труп, а наоборот, великий духовный друг наш и отец, носитель духовной благодатной жизни, молитвенник за нас пред Богом."], ["монах 0"], ["член религиозной общины, давший обет ведения аскетической жизни"], [1]]

        #создаим пустой тезнор размерности 1x2x1 для хранения эмбедингов сэмплов из батчей (итоговый будет размерности Nx2x1536,где N - число сэмплов в батче)
        embd_batch = torch.tensor([[[],[]]]).to(device)
        #необходимо для правильной установки размерности в батче embd_batch эмбедингов
        first_pass = False
        for i in range(len(labels)):
            #получаем эмбединги ключевого слова из примера употребления
            example_token_vec = self.get_vector(tokens_tensor_samp[i],segments_tensors_samp[i])
            examples_token_key_word_position = self.token_detection(offset_mapping_samp[i],samp_position_samp[i])
            example_embeddings = self.vector_recognition(example_token_vec, examples_token_key_word_position)

            #получаем эмбединги ключевого слова из определения
            def_token_vec = self.get_vector(tokens_tensor_def[i],segments_tensors_def[i])
            def_token_key_word_position = self.token_detection(offset_mapping_def[i],samp_position_def[i])
            def_embeddings = self.vector_recognition(def_token_vec, def_token_key_word_position)

            #объединяем два вектора в 1 и добавляем в общий массив (получаем тензор 2x1536)
            embd_sample = torch.stack((example_embeddings,def_embeddings)).to(device)
            if not first_pass:
              embd_batch = torch.cat((embd_batch,embd_sample.unsqueeze(0)),-1)
              first_pass = True
            else:
              embd_batch = torch.cat((embd_batch,embd_sample.unsqueeze(0)),0)

        # print(f"embd shape {embd_batch.shape}")
        x,_ = self.biderectional_lstm(embd_batch)
        # print(f"x shape {x.shape}")
        #GlobalAveragePooling1D заменяется торч мином по 1 измерению mean(dim=(1))
        Ax = torch.mean(x,1)
        #MaxAveragePooling1D заменяется  Mx,_ = torch.max(x,1)
        Mx,_ = torch.max(x,1)
        # print(f"ax shape {Ax.shape} Mx shape {Mx.shape}")
        concatenate = torch.cat((Ax, Mx), dim=1)
        # print(f"conc shape {concatenate.shape}")
        Do = self.Dropout(concatenate)
        logits = self.Linear(Do)
        loss = None

        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        #transformers.modeling_outputs.SequenceClassifierOutput используется как для обучения так
        #и для функции compute_metrics
        return transformers.modeling_outputs.SequenceClassifierOutput(
            logits=logits,
            loss=loss,
        )


    def token_detection(self, token_map, position):
        #Функция определения ключевого слова
        """
        :param token_map: list of tuples of begin and end of every token
        :param position:  list of type: [int,int]
        :return: list of key word tokens position
        """
        #из за того что в начале стоит CLS позиции начала и конца ключевого слова сдвигаются на 5
        begin_postion = position[0] #+ 5
        end_position = position[1] #+ 5

        position_of_key_tokens = []
        for token_tuple in range(1,len(token_map)-1):
            #if token is one
            if token_map[token_tuple][0] == begin_postion and token_map[token_tuple][1] == end_position:
                position_of_key_tokens.append(token_tuple)
                break

            #if we have multipli count of tokens for one key word
            if token_map[token_tuple][0] >= begin_postion and token_map[token_tuple][1] != end_position:
                position_of_key_tokens.append(token_tuple)
            if token_map[token_tuple][0] != begin_postion and token_map[token_tuple][1] == end_position:
                position_of_key_tokens.append(token_tuple)
                break

        return position_of_key_tokens
    
    def get_vector(self, tokens_tensor, segments_tensors):
        #Функция получения вектора ключевого слова
        with torch.no_grad():
            outputs = self.model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
        #from [# layers, # batches, # tokens, # features] to [# tokens, # layers, # features]
        token_dim = torch.stack(hidden_states, dim=0)
        token_dim = torch.squeeze(token_dim, dim=1)
        token_dim = token_dim.permute(1, 0, 2)
        token_vecs_cat = []
        for token in token_dim:
            cat_vec = torch.cat((token[-1], token[-2]), dim=0)
            token_vecs_cat.append(cat_vec)

        return token_vecs_cat

    def get_avarage_embedding(self,embeddings_list, positions_list):
        #Функция получения среднего вектора
        avg_tensor = torch.stack((embeddings_list[positions_list[0]],))
        for i in range(1, len(positions_list)):
              avg_tensor = torch.cat((avg_tensor,embeddings_list[positions_list[i]].unsqueeze(0)))

        average_embedding = torch.mean(avg_tensor,0)
        return average_embedding 

    def vector_recognition(self, tokens_embeddings_ex, tokens_key_word_position_ex):
        #Функция подготовки вектора в зависимости от количества токенов,которым представляется ключевое слово
        if len(tokens_key_word_position_ex) > 1:
            embeddings_data = torch.tensor(self.get_avarage_embedding(tokens_embeddings_ex,tokens_key_word_position_ex))
        else:
            # print(tokens_embeddings_ex)
            # print(tokens_key_word_position_ex)
            embeddings_data = torch.tensor(tokens_embeddings_ex[tokens_key_word_position_ex[0]])
        return embeddings_data



##Разбиение обучающей и тестовой выборок

In [108]:
from sklearn.model_selection import train_test_split

In [125]:
data = generate_bert_input('/content/drive/MyDrive/dataset_not_embeddings.json','sberbank-ai/ruBert-base')

loading file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/894ee6c75173bea620f08bc907dbb99c6da1a0cee7ce7f81c3ad91dbb0d3b2a6.8a15a3f9a238139bbd1d85ed0715ebad4078d4919f57be2a94d999cb6ff4d054
loading file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3ff2b30ffd2e83991ada1f23ca4d7adad284baa741ea21704f02d83b72405c79.072018299fd210a7d734e8e87cbe1d148c2d27f90ed55251c1d9472dc018ce32
loading

In [127]:
RANDOM_SEED=1
data_train, data_test = train_test_split(data, test_size=0.4, random_state=RANDOM_SEED)
data_val, data_test = train_test_split(data_test, test_size=0.5, random_state=RANDOM_SEED)

In [128]:
hd_train = HyperonymDataset(data_train)
hd_val = HyperonymDataset(data_val)
hd_test = HyperonymDataset(data_test)

##Задание параметров обучения

In [112]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    optim = "adamw_hf"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [113]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

##Обучение  и оценка модели

In [129]:
model = SemanticSimilarityBertModel()
model = model.to(device)

loading configuration file https://huggingface.co/sberbank-ai/ruBert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3ff2b30ffd2e83991ada1f23ca4d7adad284baa741ea21704f02d83b72405c79.072018299fd210a7d734e8e87cbe1d148c2d27f90ed55251c1d9472dc018ce32
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "u

In [130]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hd_train,
    eval_dataset=hd_val,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 480
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 120
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommend

Step,Training Loss
10,0.560700
20,0.487400
30,0.497400
40,0.500000
50,0.588000
60,0.541400
70,0.551300
80,0.473500
90,0.493100
100,0.393700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=120, training_loss=0.5109731356302897, metrics={'train_runtime': 945.3962, 'train_samples_per_second': 1.015, 'train_steps_per_second': 0.127, 'total_flos': 0.0, 'train_loss': 0.5109731356302897, 'epoch': 2.0})